By: Noah Crowley

Case ID: nwc17

## *WARNING - DO NOT RUN CELLS*
This is just analyzing my dataset and getting it into an acceptable state for later Bayesian analysis. This will still be useful to read, but if you run the cells then your computer will likely be frozen for the next half hour. It's a lot of recursive code going over 400k rows of data a lot of times, and Python is slow.

## Gathering Priors

In order to have accurate priors, it helps to have a pool of data to analyze. Thankfully, the Iowa Office of Traffic & Safety offers large CSV dataset to the public of all crash data. The unfortunate thing is that this data contains a lot of extra information that is not useful to the officer described in the main notebook of this exercise, and also that the actual priors the officer would like to have are not readily available to the public.

So, in this notebook, I will go through the process of transforming the data from my csv to my model, saving that to another csv (for easier use later on, if necessary), and then creating methods to derive priors as necessary.

In [1]:
import pandas
from pandas.core.frame import DataFrame

In [2]:
relevant_columns = [
    "DRIVERAGE", "VISIONOBS", "OCCUPANTS", "VACTION",
    "CASENUMBER", "DRUGTEST", "DRUGRESULT", "ALCRESULT",
    "TRAFCONT", "CSEVERITY", "CSURFCOND", "FATALITIES"
]
data_frame = pandas.read_csv("iowa_crash_data.csv", delimiter = ",", header = 0, usecols = relevant_columns, 
                             dtype = {"ALCRESULT": float}, error_bad_lines = False)

In [3]:
model_cols = [
    "Age group", "Number of occupants", "Impaired", "Had traffic control",
    "Surface condition", "Action", "Severity", "Number of fatalities"
]
model_data = []

for index, row in data_frame.iterrows():
    model_row = {
        "Number of occupants": row["OCCUPANTS"],
        "Impaired": False,
        "Had traffic control": row["TRAFCONT"] > 1,
        "Surface condition": row["CSURFCOND"],
        "Action": row["VACTION"],
        "Severity": row["CSEVERITY"],
        "Number of fatalities": row["FATALITIES"]
    }
    
    if row["VISIONOBS"] > 1:
        model_row["Impaired"] = True
    elif row["ALCRESULT"] > 1:
        model_row["Impaired"] = True
    elif row["DRUGTEST"] > 1:
        if row["DRUGRESULT"] > 1:
            model_row["Impaired"] = True
        
    age = row["DRIVERAGE"]
    if age < 18:
        model_row["Age group"] = 1
    elif age < 26:
        model_row["Age group"] = 2
    elif age < 65:
        model_row["Age group"] = 3
    else:
        model_row["Age group"] = 4
            
    model_data.append(model_row)

model_data_frame = pandas.DataFrame(
    data = model_data,
    columns = model_cols
)

model_data = None #Clear some memory

model_data_frame.to_csv("relevant_iowa_crash_data.csv")

Now that I have the data, I can begin getting my distributions for each column. To make this easier, I will just create a function that returns the entire distribution of any given column.

In [40]:
def get_distribution(df, col_name):
    column = df[col_name]
    value_counts = column.value_counts()
    values = column.unique()
    distribution = {}
    
    values.sort()
    
    for value in values:
        count = value_counts[value]
        distribution[value] = count / column.size
        
    return distribution

Finally, I can begin getting all of my distributions.

In [41]:
distributions = {}

for col_name in model_cols:
    distributions[col_name] = get_distribution(model_data_frame, col_name)

Now that all my distributions are calculated, I want to be able to see them. I can do this by using a function to generate tables for my data and rendering that HTML. I will use a function from [StackOverflow](https://stackoverflow.com/questions/30523735/python-dictionary-as-html-table-in-ipython-notebook) to do just that.

In [6]:
class DictTable:
    def __init__(self, dictionary, headers, title, width = 100):
        self.dictionary = dictionary
        self.headers = headers
        self.title = title
        self.width = width
        
    def _repr_html_(self):
        html = []
        
        if self.title:
            html.append("<h4>{0}</h4><br />".format(self.title))
        
        html.append("<table width={0}%>".format(self.width))
        
        html.append("<thead><tr>")
        for header in self.headers:
            html.append("<td>{0}</td>".format(header))
        html.append("</tr></thead>")
        
        html.append("<tbody>")
        for key, value in self.dictionary.items():
            html.append("<tr>")
            html.append("<td>{0}</td>".format(key))
            html.append("<td>{:1.3f}</td>".format(value))
            html.append("</tr>")
        html.append("</tbody>")
            
        html.append("</table>")
        return ''.join(html)

And now I can do this for each of my columns. I will create one more function to do this easily.

In [7]:
from IPython.display import display_html

def display_distributions():
    headers = ["Value", "Probability"]

    for col_name in model_cols:
        dict_table = DictTable(dictionary = distributions[col_name], headers = headers, title = col_name, width = 30)
        display_html(dict_table)

Now let's see the results.

In [42]:
display_distributions()

Age group Value Probability 1 0.069 2 0.214 3 0.543 4 0.174

Number of occupants Value Probability 0.0 0.049 1.0 0.687 2.0 0.182 3.0 0.050 4.0 0.021 5.0 0.007 6.0 0.002 7.0 0.001 8.0 0.000 9.0 0.000 10.0 0.000 11.0 0.000 12.0 0.000 13.0 0.000 14.0 0.000 15.0 0.000 16.0 0.000 17.0 0.000 18.0 0.000 19.0 0.000 20.0 0.000 21.0 0.000 22.0 0.000 23.0 0.000 24.0 0.000 25.0 0.000 26.0 0.000 27.0 0.000 28.0 0.000 29.0 0.000 30.0 0.000 31.0 0.000 32.0 0.000 33.0 0.000 34.0 0.000 35.0 0.000 36.0 0.000 37.0 0.000 38.0 0.000 39.0 0.000 40.0 0.000 41.0 0.000 42.0 0.000 43.0 0.000 44.0 0.000 45.0 0.000 46.0 0.000 47.0 0.000 48.0 0.000 49.0 0.000 50.0 0.000 51.0 0.000 53.0 0.000 55.0 0.000 56.0 0.000 60.0 0.000 61.0 0.000 65.0 0.000 66.0 0.000 68.0 0.000 69.0 0.000 70.0 0.000 75.0 0.000 76.0 0.000 78.0 0.000 99.0 0.000

Impaired Value Probability False 0.772 True 0.228

Had traffic control Value Probability False 0.609 True 0.391

Surface condition Value Probability 1.0 0.658 2.0 0.128 3.0 0.057 4.0 0.077 5.0 0.013 6.0 0.014 7.0 0.000 8.0 0.000 9.0 0.000 10.0 0.002 77.0 0.035 99.0 0.013 98.0 0.003

Action Value Probability 1.0 0.584 2.0 0.094 3.0 0.037 4.0 0.003 5.0 0.007 6.0 0.017 7.0 0.008 8.0 0.002 9.0 0.025 10.0 0.067 11.0 0.047 12.0 0.044 13.0 0.003 14.0 0.002 15.0 0.000 16.0 0.000 17.0 0.000 18.0 0.000 99.0 0.016 77.0 0.028 98.0 0.017

Severity Value Probability 1.0 0.006 2.0 0.023 3.0 0.089 4.0 0.172 5.0 0.710

Number of fatalities Value Probability 0.0 0.994 1.0 0.005 2.0 0.001 3.0 0.000 4.0 0.000 7.0 0.000

#### Dealing with negligible probabilities

As can be seen above, several of the values have probabilities that are shown as 0.000. I will consider any value less than 0.001 to be negligible, and will remove the value as a result. Let's transform this data now:

In [43]:
threshold = 0.001

for col_name in model_cols:
    dist = distributions[col_name]
    keys_to_remove = [k for k,v in dist.items() if v < threshold]
    
    for key in keys_to_remove:
        del dist[key]
    
    scale_factor = 1 / sum(dist.values())
    
    dist.update((key, value*scale_factor) for key, value in dist.items())

And now I'll display the data again.

In [44]:
display_distributions()

Age group Value Probability 1 0.069 2 0.214 3 0.543 4 0.174

Number of occupants Value Probability 0.0 0.049 1.0 0.689 2.0 0.182 3.0 0.050 4.0 0.021 5.0 0.007 6.0 0.002

Impaired Value Probability False 0.772 True 0.228

Had traffic control Value Probability False 0.609 True 0.391

Surface condition Value Probability 1.0 0.658 2.0 0.128 3.0 0.057 4.0 0.077 5.0 0.013 6.0 0.014 10.0 0.002 77.0 0.035 99.0 0.013 98.0 0.003

Action Value Probability 1.0 0.585 2.0 0.094 3.0 0.037 4.0 0.003 5.0 0.007 6.0 0.017 7.0 0.008 8.0 0.002 9.0 0.025 10.0 0.067 11.0 0.047 12.0 0.044 13.0 0.003 14.0 0.002 99.0 0.016 77.0 0.028 98.0 0.017

Severity Value Probability 1.0 0.006 2.0 0.023 3.0 0.089 4.0 0.172 5.0 0.710

Number of fatalities Value Probability 0.0 0.995 1.0 0.005

Unfortunately, these priors are not exactly all very useful. I also need the conditional probability distributions of all of my variables according to the hierarchy, which is a more complicated request.

In [148]:
from pandas.core.frame import DataFrame

#This is adapted from Morgan's demo
def calc_cond_prob(trace, event, num_decimals = 3):
    total_trace_size = len(trace)
    if total_trace_size == 0:
        return 0.0
    for k, v in event.items():
        trace = [smp for smp in trace if smp[k] == v]
    return round(len(trace) / total_trace_size, num_decimals)

#This creates the condition data efficiently
def create_cond_data(df, existing_trace, other_conds, remaining_givens):
    if type(existing_trace) is DataFrame:
        existing_trace = existing_trace.transpose().to_dict().values()
        
    cond_data = []
    
    given_col_name = remaining_givens[0]
    given_values = df[given_col_name].unique()
    given_values.sort()
    
    for given_value in given_values:
        cur_cond = dict(other_conds)
        cur_cond[given_col_name] = given_value
        cur_trace = [smp for smp in existing_trace if smp[given_col_name] == given_value]
        if (len(remaining_givens) == 1): #This is the last column name
            cond_data.append({
                "name": str(cur_cond),
                "trace": cur_trace,
                "condition": cur_cond
            })
        else: #There are more columns to go
            cond_data.extend(create_cond_data(df, cur_trace, cur_cond, remaining_givens[1:]))
            
    return cond_data

#This finally just computes the cpd of a variable given some set of other variables
def compute_cpd(df, event_col_name, given_col_names):
    cpd = {
        "evidence": given_col_names,
        "probabilities": {}
    }
    cond_data = create_cond_data(df, df, {}, given_col_names)
    
    for event_value in df[event_col_name].unique():
        cpd["probabilities"][event_value] = {}
        for cond in cond_data:
            cpd["probabilities"][event_value][cond["name"]] = calc_cond_prob(cond["trace"], {event_col_name: event_value})
            
    return cpd

In [216]:
a_cpd = compute_cpd(model_data_frame, "Action", ["Age group", "Had traffic control", "Surface condition", "Number of occupants", "Impaired"])

In [217]:
s_cpd = compute_cpd(model_data_frame, "Severity", ["Action", "Impaired"])

In [218]:
f_cpd = compute_cpd(model_data_frame, "Number of fatalities", ["Severity", "Number of occupants"])

Having this, I need a way to translate these computed values into a pgmpy TabularCPD. This is going to require another recursive method.

In [222]:
def get_prob_list_from_cpd(df, cpd, value, cond, remaining_givens):
    prob_list = []
    given_col_name = remaining_givens[0]
    given_values = df[given_col_name].unique()
    given_values.sort()
    
    for given_value in given_values:
        cur_cond = dict(cond)
        cur_cond[given_col_name] = given_value
        if (len(remaining_givens) == 1): #This is the last column name
            prob_list.append(cpd["probabilities"][value][str(cur_cond)])
        else: #There are more columns to go
            prob_list.extend(get_prob_list_from_cpd(df, cpd, value, cur_cond, remaining_givens[1:]))
    return prob_list

def print_pgmpy_tcpd_init(df, cpd, event_name):
    tcpd_init_str = "tcpd("
    
    event_values = df[event_name].unique()
    event_values.sort()
    
    tcpd_init_str += "variable='{}', variable_card={}, ".format(event_name, len(event_values))
    
    evidence_card = []
    for evidence_col_name in cpd["evidence"]:
        evidence_card.append(len(df[evidence_col_name].unique()))
        
    tcpd_init_str += "evidence={}, evidence_card={}, ".format(str(cpd["evidence"]), str(evidence_card))
    
    event_values_string = "[\n"
    for event_value in event_values:
        prob_list = get_prob_list_from_cpd(df, cpd, event_value, {}, cpd["evidence"])
        event_values_string += str(prob_list)
        if event_value != event_values[-1]:
            event_values_string += ","
        event_values_string += "\n"
    event_values_string += "]"
    
    tcpd_init_str += "values={}".format(event_values_string)
    tcpd_init_str += ")"
    
    print(tcpd_init_str)

Also, due to rounding error, I need to create some methods to make small changes so as to get my values to add up to 1 for each condition. As for conditions that had no data, which will all currently have a 0.0 probability, I will assume a uniform distribution of each possible event value as there is no information to draw from.

In [214]:
def get_prob_sums(df, cpd, event_name):
    event_values = df[event_name].unique()
    event_values.sort()
    
    prob_sums = get_prob_list_from_cpd(df, cpd, event_values[0], {}, cpd["evidence"])
    for event_value in event_values[1:]:
        prob_list = get_prob_list_from_cpd(df, cpd, event_value, {}, cpd["evidence"])
        prob_sums = [x + y for x,y in zip(prob_sums, prob_list)]
    
    return prob_sums

def get_prob_sums_dict(df, cpd, event_name):
    event_values = df[event_name].unique()
    event_values.sort()
    
    prob_sums = dict(cpd["probabilities"][event_values[0]])
    for event_value in event_values[1:]:
        probs = cpd["probabilities"][event_value]#get_prob_list_from_cpd(df, cpd, event_value, {}, cpd["evidence"])
        for cond_str in probs:
            prob_sums[cond_str] += probs[cond_str]
    
    return prob_sums

def print_sum_of_probs(df, cpd, event_name):
    print(get_prob_sums(df, cpd, event_name))
    
import math
def get_cond_from_index(df, given_col_names, index):
    cond = {}
    num_given_col_names = len(given_col_names)
    
    lower_conds_card_sums = [0]
    for i in range(num_given_col_names - 1,-1,-1):
        given_col_name = given_col_names[i]
        given_values = df[given_col_name].unique()
        lower_conds_card_sum = len(given_values) + lower_conds_card_sums[0]
        lower_conds_card_sums.insert(0, lower_conds_card_sum)
    lower_conds_card_sums[-1] = 1 #Helps to avoid if statements below
    
    for i in range(num_given_col_names):
        given_col_name = given_col_names[i]
        given_values = df[given_col_name].unique()
        given_values.sort()
        lower_conds_card_sum = lower_conds_card_sums[i]
        
        given_value_index = math.floor(index / lower_conds_card_sum)
            
        cond[given_col_name] = given_values[given_value_index]
        index -= given_value_index * len(given_values)
        
    return cond

#Unfortunately the math led to a lot of the values not summing to 1. This will just fix that.
def fix_prob_sums(df, cpd, event_name, num_decimals = 3):
    event_values = df[event_name].unique()
    event_values.sort()
        
    #For filling in 0.0s since those mean we have no data on it, so just assume all events are equally likely
    uniform_value = round(1 / len(event_values), num_decimals)
    
    cond_prob_sums = get_prob_sums_dict(df, cpd, event_name)
    
    for cond_str in cond_prob_sums:
        prob_sum = cond_prob_sums[cond_str]
        #prob_sum = prob_sums[i]
        #cond = get_cond_from_index(df, cpd["evidence"], i)
        #cond_str = str(cond)
        
        if prob_sum == 0.0:
            for event_value in event_values:
                cpd["probabilities"][event_value][cond_str] = uniform_value
                prob_sum += uniform_value
                
        diff = round(1.0 - prob_sum, num_decimals)
        if diff != 0.000:
            for event_value in event_values:
                if diff < 0.0 and cpd["probabilities"][event_value][cond_str] < abs(diff):
                    continue
                else:
                    cpd["probabilities"][event_value][cond_str] += diff
                    break

In [219]:
fix_prob_sums(model_data_frame, a_cpd, "Action")
fix_prob_sums(model_data_frame, s_cpd, "Severity")
fix_prob_sums(model_data_frame, f_cpd, "Number of fatalities")

In [223]:
print_sum_of_probs(model_data_frame, a_cpd, "Action")

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0000000000000002, 1.0000000000000002, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0, 1.0000000000000002, 1.0, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.000000000000

In [224]:
print_pgmpy_tcpd_init(model_data_frame, a_cpd, "Action")

tcpd(variable='Action', variable_card=21, evidence=['Age group', 'Had traffic control', 'Surface condition', 'Number of occupants', 'Impaired'], evidence_card=[4, 2, 13, 66, 2], values=[
[0.428, 0.0, 0.707, 0.586, 0.649, 0.597, 0.664, 0.52, 0.655, 0.548, 0.736, 0.501, 0.84, 0.334, 0.6, 1.0, 1.0, 0.04, 0.04, 0.04, 0.04, 0.04, 0.0, 0.04, 0.0, 0.04, 0.04, 0.04, 0.04, 0.0, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.71

In [225]:
print_sum_of_probs(model_data_frame, s_cpd, "Severity")

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [226]:
print_pgmpy_tcpd_init(model_data_frame, s_cpd, "Severity")

tcpd(variable='Severity', variable_card=5, evidence=['Action', 'Impaired'], evidence_card=[21, 2], values=[
[0.007, 0.015, 0.002, 0.003, 0.001, 0.004, 0.009000000000000001, 0.0, 0.009, 0.038, 0.002, 0.009, 0.001, 0.014, 0.024, 0.057, 0.002, 0.0, 0.002, 0.002, 0.001, 0.0, 0.001, 0.002, 0.005, 0.008, 0.024, 0.129, 0.009999999999999998, 0.0, 0.001, 0.0, 0.0, 0.041, 0.0, 0.0, 0.0, 0.0, 0.007, 0.016, 0.015, 0.003],
[0.029, 0.03, 0.022, 0.025, 0.011, 0.013, 0.025, 0.018, 0.04, 0.053, 0.015, 0.012, 0.015, 0.009, 0.035, 0.046, 0.004, 0.004, 0.014, 0.019, 0.012, 0.015, 0.007, 0.007, 0.045, 0.01, 0.067, 0.118, 0.011, 0.0, 0.045, 0.0, 0.045, 0.042, 0.0, 0.0, 0.013, 0.003, 0.034, 0.038, 0.031, 0.005],
[0.106, 0.085, 0.098, 0.104, 0.07, 0.085, 0.097, 0.123, 0.108, 0.091, 0.048, 0.053, 0.068, 0.06, 0.156, 0.129, 0.018, 0.028, 0.072, 0.108, 0.071, 0.101, 0.046, 0.038, 0.068, 0.064, 0.203, 0.212, 0.151, 0.0, 0.156, 0.267, 0.112, 0.042, 0.024, 0.0, 0.035, 0.018, 0.108, 0.104, 0.099, 0.031],
[0.188, 0.1

In [227]:
print_sum_of_probs(model_data_frame, f_cpd, "Number of fatalities")

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [228]:
print_pgmpy_tcpd_init(model_data_frame, f_cpd, "Number of fatalities")

tcpd(variable='Number of fatalities', variable_card=6, evidence=['Severity', 'Number of occupants'], evidence_card=[5, 66], values=[
[0.0, 0.0, 0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.165, 0.0, 0.0, 0.165, 0.165, 0.0, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.0, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.0, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.165, 0.165, 0.165, 1.0, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 1.0, 0.165, 1.0, 1.0, 0.165, 0.165, 0.165, 1.0, 1.0, 0.165, 0.165, 0.165, 1.0, 0.165, 0.165, 1.0, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 1.0, 1.0, 

And now, at long last, I can port this into the main notebook for this project.

#### Oh

Oh wait... pgmpy already had a method for most of this notebook.
fun.